In [1]:
# Importing the libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [2]:
Data = pd.read_csv('Data\hotelbookingdata.csv')
Data.head()

,addresscountryname,city_actual,rating_reviewcount,center1distance,center1label,center2distance,center2label,neighbourhood,price,price_night,...,accommodationtype,guestreviewsrating,scarce_room,hotel_id,offer,offer_cat,year,month,weekend,holiday
0,Netherlands,Amsterdam,1030.0,3.1 miles,City centre,3.6 miles,Montelbaanstoren,Amsterdam,172,price for 1 night,...,_ACCOM_TYPE@Hotel,4.3 /5,0,1.0,0,0% no offer,2017,11,1,0
1,Netherlands,Amsterdam,1030.0,3.1 miles,City centre,3.6 miles,Montelbaanstoren,Amsterdam,122,price for 1 night,...,_ACCOM_TYPE@Hotel,4.3 /5,0,1.0,1,15-50% offer,2018,1,1,0
2,Netherlands,Amsterdam,1030.0,3.1 miles,City centre,3.6 miles,Montelbaanstoren,Amsterdam,122,price for 1 night,...,_ACCOM_TYPE@Hotel,4.3 /5,0,1.0,1,15-50% offer,2017,12,0,1
3,Netherlands,Amsterdam,1030.0,3.1 miles,City centre,3.6 miles,Montelbaanstoren,Amsterdam,552,price for 4 nights,...,_ACCOM_TYPE@Hotel,4.3 /5,0,1.0,1,1-15% offer,2017,12,0,1
4,Netherlands,Amsterdam,1030.0,3.1 miles,City centre,3.6 miles,Montelbaanstoren,Amsterdam,122,price for 1 night,...,_ACCOM_TYPE@Hotel,4.3 /5,0,1.0,1,15-50% offer,2018,2,1,0


In [3]:
# Loading the GDP data for filtering
gdp_df = pd.read_csv('Data\GDP-Capita.csv')
gdp_df.head()

,Country Name,Country Code,2020,Region
0,Albania,ALB,5343.037704,Europe & Central Asia
1,Andorra,AND,37207.222000,Europe & Central Asia
2,Armenia,ARM,4505.867746,Europe & Central Asia
3,Austria,AUT,48809.226880,Europe & Central Asia
4,Azerbaijan,AZE,4229.910649,Europe & Central Asia


In [4]:
# Describing the column 2020:
gdp_df['2020'].describe()

count        57.000000
mean      34478.285091
std       37468.485426
min         852.330230
25%        7677.369141
50%       22242.406420
75%       49169.719340
max      182537.304600
Name: 2020, dtype: float64

As we can see 25% of the countries in this catagory have a GDP per Capita higher than 50000$.
These countries can be considered more similar in some ways and merged to a group.

In [5]:
# mapping the GDP data to the hotel booking data
Data['GDP'] = Data['addresscountryname'].map(gdp_df.set_index('Country Name')['2020'])
Data.head()

,addresscountryname,city_actual,rating_reviewcount,center1distance,center1label,center2distance,center2label,neighbourhood,price,price_night,...,guestreviewsrating,scarce_room,hotel_id,offer,offer_cat,year,month,weekend,holiday,GDP
0,Netherlands,Amsterdam,1030.0,3.1 miles,City centre,3.6 miles,Montelbaanstoren,Amsterdam,172,price for 1 night,...,4.3 /5,0,1.0,0,0% no offer,2017,11,1,0,52162.57012
1,Netherlands,Amsterdam,1030.0,3.1 miles,City centre,3.6 miles,Montelbaanstoren,Amsterdam,122,price for 1 night,...,4.3 /5,0,1.0,1,15-50% offer,2018,1,1,0,52162.57012
2,Netherlands,Amsterdam,1030.0,3.1 miles,City centre,3.6 miles,Montelbaanstoren,Amsterdam,122,price for 1 night,...,4.3 /5,0,1.0,1,15-50% offer,2017,12,0,1,52162.57012
3,Netherlands,Amsterdam,1030.0,3.1 miles,City centre,3.6 miles,Montelbaanstoren,Amsterdam,552,price for 4 nights,...,4.3 /5,0,1.0,1,1-15% offer,2017,12,0,1,52162.57012
4,Netherlands,Amsterdam,1030.0,3.1 miles,City centre,3.6 miles,Montelbaanstoren,Amsterdam,122,price for 1 night,...,4.3 /5,0,1.0,1,15-50% offer,2018,2,1,0,52162.57012


In [6]:
# Checking for null values in gdp column
print(Data['GDP'].isnull().sum())
# Checking which countries have a missing GDP value
missing_gdp =Data[Data['GDP'].isnull()]['addresscountryname'].unique()
print(missing_gdp)

26032
['Slovakia' 'Cameroon' 'Turkey' 'Egypt' 'Russia' 'Czech Republic' 'Malta']


In [7]:
# Checking to find the similar countries in the gdp data
for i in missing_gdp:
    print(i)
    print(gdp_df[gdp_df['Country Name'].str.contains(i[:5])]['Country Name'].unique()) 

Slovakia
['Slovak Republic']
Cameroon
[]
Turkey
[]
Egypt
[]
Russia
['Russian Federation']
Czech Republic
['Czechia']
Malta
[]


As we can see, for some countries (Slovakia, Russia, Czech Republic) we were able to find the equivalent country names in world bank dataset. 
We replace the missing gdp value in _Data_ df with the values from __gdp_df__ and equivalent countries. 

In [8]:
# Replacing the missing gdp values in Data with the values from gdp_df and similar countries
for i in missing_gdp:
    if len(gdp_df[gdp_df['Country Name'].str.contains(i[:5])]['2020'].values) > 0:
        Data.loc[Data['addresscountryname'] == i, 'GDP'] = gdp_df[gdp_df['Country Name'].str.contains(i[:5])]['2020'].values[0]

## Data Filtering based on the GDP
* In this project we plan to predict to hotel prices in european countries which have a gdp per capita higher than 50000$. We filter the data considering this criteria.

In [9]:
# Filtering the data based on the countries with GDP values higher than 50000$
df = Data[Data['GDP'] > 50000]
df.shape, df.head()

((7394, 25),
   addresscountryname city_actual  rating_reviewcount center1distance  \
 0        Netherlands   Amsterdam              1030.0       3.1 miles   
 1        Netherlands   Amsterdam              1030.0       3.1 miles   
 2        Netherlands   Amsterdam              1030.0       3.1 miles   
 3        Netherlands   Amsterdam              1030.0       3.1 miles   
 4        Netherlands   Amsterdam              1030.0       3.1 miles   
 
   center1label center2distance      center2label neighbourhood  price  \
 0  City centre       3.6 miles  Montelbaanstoren     Amsterdam    172   
 1  City centre       3.6 miles  Montelbaanstoren     Amsterdam    122   
 2  City centre       3.6 miles  Montelbaanstoren     Amsterdam    122   
 3  City centre       3.6 miles  Montelbaanstoren     Amsterdam    552   
 4  City centre       3.6 miles  Montelbaanstoren     Amsterdam    122   
 
           price_night  ... guestreviewsrating  scarce_room  hotel_id  offer  \
 0   price for 1 nigh

## Data Cleaning
In the process of data cleaning we need to pave the following steps:
* __Inspect for Missing Values__: Check if there are any missing values in the columns that we intend to use.
* __Handle Categorical Data__: Convert categorical columns like city_actual, neighbourhood, etc., into a format suitable for machine learning models (e.g., through one-hot encoding).
* __Handle Numerical Data__: Convert distances from string to numerical format (e.g., center1distance). Also, scale or normalize numerical features if required.
* __Feature Selection and Labels__: Decide which columns will be used as input and which column will be the output.

In [130]:
# Checking the missing values
missing_values = df.isnull().sum()

# Checking the data types and unique values
data_types = df.dtypes
unique_values = df.nunique()

# Summarizing the above results
Data_summary = pd.DataFrame({'Data types': data_types, 'Unique values': unique_values, 'Missing values': missing_values})
Data_summary.sort_values(by = 'Missing values', ascending = False)

,Data types,Unique values,Missing values
rating2_ta,float64,8,400
rating2_ta_reviewcount,float64,773,400
rating_reviewcount,float64,604,189
guestreviewsrating,object,25,189
holiday,int64,2,0
weekend,int64,2,0
month,int64,8,0
year,int64,2,0
offer_cat,object,5,0
offer,int64,2,0


In [17]:
# Extracting the guest score and converting it to float
df['guestreviewsrating'] = df['guestreviewsrating'].str.extract('(\d+)')
df['guestreviewsrating'] = df['guestreviewsrating'].astype(float)

C:\Users\PA\AppData\Local\Temp\ipykernel_19080\249034060.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['guestreviewsrating'] = df['guestreviewsrating'].str.extract('(\d+)')
C:\Users\PA\AppData\Local\Temp\ipykernel_19080\249034060.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['guestreviewsrating'] = df['guestreviewsrating'].astype(float)


In [19]:
# replacing the missing values in the rating columns with the mode values
df['rating2_ta'].fillna(df['rating2_ta'].mean(), inplace = True)
df['rating2_ta_reviewcount'].fillna(df['rating2_ta_reviewcount'].mean(), inplace = True)
df['rating_reviewcount'].fillna(df['rating_reviewcount'].mean(), inplace = True)
df['guestreviewsrating'].fillna(df['guestreviewsrating'].mean(), inplace = True)

C:\Users\PA\AppData\Local\Temp\ipykernel_19080\3504336932.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['rating2_ta'].fillna(df['rating2_ta'].mean(), inplace = True)
C:\Users\PA\AppData\Local\Temp\ipykernel_19080\3504336932.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['rating2_ta_reviewcount'].fillna(df['rating2_ta_reviewcount'].mean(), inplace = True)
C:\Users\PA\AppData\Local\Temp\ipykernel_19080\3504336932.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/in

In [20]:
df.isnull().sum()

addresscountryname        0
city_actual               0
rating_reviewcount        0
center1distance           0
center1label              0
center2distance           0
center2label              0
neighbourhood             0
price                     0
price_night               0
s_city                    0
starrating                0
rating2_ta                0
rating2_ta_reviewcount    0
accommodationtype         0
guestreviewsrating        0
scarce_room               0
hotel_id                  0
offer                     0
offer_cat                 0
year                      0
month                     0
weekend                   0
holiday                   0
GDP                       0
dtype: int64

# Feature Selection & ِData Cleaning
In this step we are going to decide which variables to use in our prediction model.
* Our target variable is _"Price/Night"_. We can derive this variable by dividing the price to __price_night__.
* It seems trivial that accommodatin_type affects the price. (Type = Categorical)
* Location variables (Country, City): Location is a key factor in pricing. (Type = Categorical)
* Star_rating
* rating, rating count
* ratingta (advisor rating)
* Distance, Distance alter
* year, month
* scarce room

In [132]:
df.head()

,addresscountryname,city_actual,rating_reviewcount,center1distance,center1label,center2distance,center2label,neighbourhood,price,price_night,...,guestreviewsrating,scarce_room,hotel_id,offer,offer_cat,year,month,weekend,holiday,GDP
0,Netherlands,Amsterdam,1030.0,3.1 miles,City centre,3.6 miles,Montelbaanstoren,Amsterdam,172,price for 1 night,...,4.3 /5,0,1.0,0,0% no offer,2017,11,1,0,52162.57012
1,Netherlands,Amsterdam,1030.0,3.1 miles,City centre,3.6 miles,Montelbaanstoren,Amsterdam,122,price for 1 night,...,4.3 /5,0,1.0,1,15-50% offer,2018,1,1,0,52162.57012
2,Netherlands,Amsterdam,1030.0,3.1 miles,City centre,3.6 miles,Montelbaanstoren,Amsterdam,122,price for 1 night,...,4.3 /5,0,1.0,1,15-50% offer,2017,12,0,1,52162.57012
3,Netherlands,Amsterdam,1030.0,3.1 miles,City centre,3.6 miles,Montelbaanstoren,Amsterdam,552,price for 4 nights,...,4.3 /5,0,1.0,1,1-15% offer,2017,12,0,1,52162.57012
4,Netherlands,Amsterdam,1030.0,3.1 miles,City centre,3.6 miles,Montelbaanstoren,Amsterdam,122,price for 1 night,...,4.3 /5,0,1.0,1,15-50% offer,2018,2,1,0,52162.57012


### Defining the Target Variable
* For this task we need to convert the __price_night__ variable to numerical values.

In [133]:
import re
# Extract numerical values from "price_night" column
df['price_night'] = df['price_night'].apply(lambda x: re.findall(r'\d+', str(x))[0])

# Convert the extracted values to numeric type
df['price_night'] = pd.to_numeric(df['price_night'])

# Creating target variable: price per night
df['price_per_night'] = df['price']/df['price_night']

C:\Users\PA\AppData\Local\Temp\ipykernel_11904\1758452389.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['price_night'] = df['price_night'].apply(lambda x: re.findall(r'\d+', str(x))[0])
C:\Users\PA\AppData\Local\Temp\ipykernel_11904\1758452389.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['price_night'] = pd.to_numeric(df['price_night'])
C:\Users\PA\AppData\Local\Temp\ipykernel_11904\1758452389.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

### Accommodation Type

In [134]:
print(df['accommodationtype'].unique())

['_ACCOM_TYPE@Hotel' '_ACCOM_TYPE@Hostel' '_ACCOM_TYPE@Apartment'
 '_ACCOM_TYPE@Apart-hotel' '_ACCOM_TYPE@House boat' '_ACCOM_TYPE@Inn'
 '_ACCOM_TYPE@Guest House' '_ACCOM_TYPE@Bed and breakfast'
 '_ACCOM_TYPE@Castle' '_ACCOM_TYPE@Vacation home Condo'
 '_ACCOM_TYPE@Resort' '_ACCOM_TYPE@']


In [135]:
df.loc[:, 'accommodationtype'] = df['accommodationtype'].str.split('@').str[1]

C:\Users\PA\AppData\Local\Temp\ipykernel_11904\2666905615.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.loc[:, 'accommodationtype'] = df['accommodationtype'].str.split('@').str[1]


In [136]:
df['accommodationtype'].unique()

array(['Hotel', 'Hostel', 'Apartment', 'Apart-hotel', 'House boat', 'Inn',
       'Guest House', 'Bed and breakfast', 'Castle',
       'Vacation home Condo', 'Resort', ''], dtype=object)

In [137]:
# dropping the observations with df['accommodationtype']==''
df = df[df['accommodationtype'] != '']

In [138]:
df['accommodationtype'].unique()

array(['Hotel', 'Hostel', 'Apartment', 'Apart-hotel', 'House boat', 'Inn',
       'Guest House', 'Bed and breakfast', 'Castle',
       'Vacation home Condo', 'Resort'], dtype=object)

### Center Labels and Center Distances

In [139]:
df['center1label'].unique()

array(['City centre'], dtype=object)

In [140]:
df['center2label'].unique()

array(['Montelbaanstoren', 'City Safari', 'Clontarf Castle',
       'Port of Rotterdam', 'Eriksdalsbadet'], dtype=object)

* Center Labels do not seem to be very useful. We can drop them.

In [141]:
# Dropping the columns "center1label" and "center2label"
df.drop(['center1label', 'center2label'], axis = 1, inplace = True)

In [142]:
# Extracting the float values from the "center1distance" and "center2distance" columns
df['center1distance'] = df['center1distance'].apply(lambda x: re.findall(r'\d+\.\d+', str(x))[0] if re.findall(r'\d+\.\d+', str(x)) else None)
df['center2distance'] = df['center2distance'].apply(lambda x: re.findall(r'\d+\.\d+', str(x))[0] if re.findall(r'\d+\.\d+', str(x)) else None)
# Converting the extracted values to numeric type
df['center1distance'] = pd.to_numeric(df['center1distance'])
df['center2distance'] = pd.to_numeric(df['center2distance'])

## Ratings
* Guest Review

In [143]:
# Extracting the float values from the "guestreviewsrating" column before the " /"
df['guestreviewsrating'] = df['guestreviewsrating'].apply(lambda x: re.findall(r'\d+\.\d+', str(x))[0] if re.findall(r'\d+\.\d+', str(x)) else None)
# converting the extracted values to numeric type
df['guestreviewsrating'] = pd.to_numeric(df['guestreviewsrating'])
#droping the observations with df['guestreviewsrating'] == None
df = df[df['guestreviewsrating'].notna()]
df['guestreviewsrating'].unique()

array([4.3, 4.1, 3.5, 4.4, 4.8, 3.7, 3.9, 4.5, 2.9, 4.6, 2.7, 3.4, 3.2,
       2.4, 4.9, 2.2, 2.5, 1.7, 1.3, 1.2])

### Offer_cat
* This feature is a ordinal categorical variable and it seems the order of categories matters a lot in the pricing process. So, we need to use Label Encoding method to convert it to a variable that can be used in a regression model.

In [144]:
df['offer_cat'].unique()

array(['0% no offer', '15-50% offer', '1-15% offer', '50%-75% offer',
       '75%+ offer '], dtype=object)

In [145]:
# Convert the offer_cat column to categorical type using LabelEncoder
ordered_categories = {'0% no offer': 0, r'15-50% offer':1, r'1-15% offer':2, r'50%-75% offer':3, r'75%+ offer ':4}
df['offer_cat'] = df['offer_cat'].map(ordered_categories)

In [146]:
df['offer_cat'].unique()

array([0, 1, 2, 3, 4], dtype=int64)